# Basic demonstration of Dspy integration functionality with the Unity Catalog AI Toolkit SDK

To get started with this, you will need an OpenAI API Key. 

Once you have acquired your key, set it to the environment variable `OPENAI_API_KEY` after storing it in the `Databricks Secrets` API (accessible via dbutils or the databricks sdk workspace client).

In [ ]:
%pip install -Uqqq unitycatalog-dspy[databricks] dspy mlflow

%restart_python

## Setting your API Key

Don't forget to remove the key after you're done running cell 4!

In [ ]:
import base64
import os

from databricks.sdk import WorkspaceClient

workspace_client = WorkspaceClient()

secret_scope = "puneet_jain"  # Change me!

# Run this if you don't have the API key set to your secrets scope yet

# if secret_scope not in [scope.name for scope in workspace_client.secrets.list_scopes()]:
#     workspace_client.secrets.create_scope(secret_scope)

# my_secret = "<your API key, temporarily>"

# workspace_client.secrets.put_secret(scope=secret_scope, key="openai_api_key", string_value=my_secret)

## Fetch the key and set it to the environment variable key that the OpenAI SDK needs

In [39]:
os.environ["OPENAI_API_KEY"] = base64.b64decode(
    workspace_client.secrets.get_secret(scope=secret_scope, key="openai_api_key").value
).decode()

In [41]:
assert "OPENAI_API_KEY" in os.environ, (
    "Please set the OPENAI_API_KEY environment variable to your OpenAI API key"
)

In [26]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
from unitycatalog.ai.dspy.toolkit import UCFunctionToolkit

## Import the UC client for Databricks UC. 
This will allow for function creation through either the `create_function` API (requires the defined `sql_body` statement) or the `create_python_function` (requires a type-hint-applied and docstring commented python callable). 

In [27]:
CATALOG = "mlops_pj"  # Change me !
SCHEMA = "rr_rag_chatbot"  # Change me if you want

In [31]:
client = DatabricksFunctionClient()

### Define a Callable
The requirements for the callable:

**typing**

Types **must** be supplied for both the arguments and the return type. Function signatures that do not have these defined will raise a `ValueError`.

The following types are not allowed:
`Union`
`Any`

Additional caveats:
Collections **must** supply typing of the interior components. For instance, ``typing.Dict`` is not allowed, but ``typing.Dict[str, str]`` will work correctly. 

**doc strings**

The doc string **must** be in the Google Docstring format.
Args and Returns comments are optional, but the function description **is required**. 

In [32]:
def get_weather(city: str) -> str:
    """Retrieve mock weather information for a given city.

    This function looks up predefined mock weather data for a set of cities.
    If the city is not found in the dataset, a default message is returned.

    Args:
        city (str): The name of the city to retrieve weather data for.

    Returns:
        str: A string describing the weather for the given city, or
        "Weather data not available" if the city is not in the dataset.

    Example:
        >>> get_weather("New York")
        'Sunny, 25°C'

        >>> get_weather("Boston")
        'Weather data not available'
    """
    mock_data = {
        "New York": "Sunny, 25°C",
        "Los Angeles": "Cloudy, 20°C",
        "Chicago": "Rainy, 15°C",
        "Houston": "Thunderstorms, 30°C",
        "Phoenix": "Sunny, 35°C",
    }
    return mock_data.get(city, "Weather data not available")

In [33]:
client.create_python_function(func=get_weather, catalog=CATALOG, schema=SCHEMA, replace=True)

FunctionInfo(browse_only=None, catalog_name='mlops_pj', comment='Retrieve mock weather information for a given city. This function looks up predefined mock weather data for a set of cities. If the city is not found in the dataset, a default message is returned.', created_at=1756333139900, created_by='puneet.jain@databricks.com', data_type=<ColumnTypeName.STRING: 'STRING'>, external_language='Python', external_name=None, full_data_type='STRING', full_name='mlops_pj.rr_rag_chatbot.get_weather', function_id='e1c1b67c-82ff-4ce9-8501-ba194bcce21e', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='city', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>, position=0, comment='The name of the city to retrieve weather data for.', parameter_default=None, parameter_mode=None, parameter_type=<FunctionParameterType.PARAM: 'PARAM'>, type_interval_type=None, type_json='{"name":"city","type":"string","nullable":true,"metadata":{"comment":"The name of the city t

In [34]:
# Create a tool instance to use with Dspy

toolkit = UCFunctionToolkit(function_names=[f"{CATALOG}.{SCHEMA}.get_weather"], client=client)

## Enable tracing in MLflow
Auto-enabling tracing allows us to see the calls made by LlamaIndex

In [ ]:
import mlflow

mlflow.dspy.autolog()

In [35]:
import dspy

llm = dspy.LM(model="openai/gpt-4o", api_key=os.getenv("OPENAI_API_KEY"))
dspy.configure(lm=llm)

### Build a simple ReAct agent
We define a DSPy signature (`Agent_tools`) for the agent’s inputs/outputs and compose a ReAct module with `tools=toolkit.tools`. The agent maintains a minimal chat history and can call the UC function when helpful.


In [36]:
# Create a DSPy signature for the agent's inputs/outputs
class Agent_tools(dspy.Signature):
    """AI assistant Able to use tools"""

    input_query = dspy.InputField()
    history = dspy.InputField(desc="Chat History")
    output = dspy.OutputField(desc="Response to User")


# Create a DSPy module for the agent with  ReAct Framework and Tool Calling
class Agent(dspy.Module):
    def __init__(self):
        super().__init__()
        self.history = []
        self.tools_utils = dspy.ReAct(Agent_tools, tools=toolkit.tools)

    def __call__(self, input: str):
        self.add_memory(input)
        str_history = "\n".join(self.history.copy())
        result = self.tools_utils(input_query=input, history=str_history).output
        self.add_memory(result)
        return result

    def add_memory(self, memory: str):
        self.history.append(("USER: " if len(self.history) % 2 == 0 else "AI: ") + memory)

In [37]:
chat_module = Agent()
human_input = "what is the weather in new york and chicago?"
print("AI: ", chat_module(human_input))

AI:  The current weather in New York is Sunny with a temperature of 25°C. In Chicago, it is Rainy with a temperature of 15°C.
